# Settle Airbnb Data Analysis

## What kind of information am I looking for?

- What are the busiest times of the year to visit Seattle? How will the price react to it?
- Is there a general upward trend of both new Airbnb listings and total Airbnb visitors to Seattle?
- How to be successful in Airbnb business?

In [ ]:
# Extra
# predict positive / negative comment? sentiment - https://docs.aws.amazon.com/comprehend/latest/dg/how-sentiment.html
# find out the variables relationships

In [ ]:
# import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

First, we take a look at the data

In [ ]:
# explore calendar data
df_cal = pd.read_csv('./seattle/calendar.csv')
df_cal.head()

In [ ]:
# explore listing data
df_listings = pd.read_csv('./seattle/listings.csv')
df_listings.head()

In [ ]:
# explore reviews data
df_reviews = pd.read_csv('./seattle/reviews.csv')
df_reviews.head()

In [ ]:
print('calendar shape:', df_cal.shape)
print('listings shape:', df_listings.shape)
print('reviews shape:', df_reviews.shape)

## Data Preparation

As amenities are packed in 1 field, we need to split them to different columns and change the value to 1 or 0

In [ ]:
# transform amenities values to one or zero

amenities = []

for val in df_listings.amenities:
    try:
        # remove brackets
        val =  val[1:-1]
        # remove quote sign
        val = val.replace("\"", "")
        amenities.extend(val.split(','))
    except AttributeError:
        pass

# convert to set
amenities = set(amenities)

# remove empty string
amenities.remove('')
print(amenities)

In [ ]:
def split_amenities(val, amenity_check):
    """
    Function to check if ta string contains the amenity string and convert to 1 and 0
    
    input:  
    val - (string) value to check if it contains the amenity string
    amenity_check - (string) the amenity string to check
    
    output: 
    (int) if val contains the amennity, return 1; return 0 otherwise
    """
    try:
        if val.find(amenity_check) >-1:
            return 1
        else:
            return 0
    except AttributeError:
        return 0

# convert values    
for amenity in amenities:        
    df_listings[amenity] = df_listings['amenities'].apply(lambda x: split_amenities(x, amenity))
        
df_listings.head()    

In [ ]:
# check missing data for calendar
np.sum(df_cal.isnull()).sort_values(ascending=False)

In [ ]:
# check why the price is missing
df_cal[df_cal['price'].isnull()].groupby('available').count()

In calendar data, as we can see, the price is missing when "available" is "f". This is normal as when the listing is not available for booking, there should be no price. No rows will be removed.

In [ ]:
# check missing data for listins
df_listings_null_count = np.sum(df_listings.isnull()).rename_axis('column').reset_index(name='counts')
df_listings_null_count = df_listings_null_count[df_listings_null_count['counts'] > 0]
df_listings_null_count.sort_values(by='counts', ascending=False)

In listing data, we can see some columns are with na / null values. However, as we will mainly use count and mean to perform the analysis (NA values are excluded by default), we will not remove an rows.

In [ ]:
#df_listings.hist()

## Q1: What are the busiest times of the year to visit Seattle? How will the price react to it?

In [ ]:
# convert date to datetime
df_cal['date'] = pd.to_datetime(df_cal['date'])

# check the busiest period
df_cal['month'] = df_cal['date'].dt.to_period('M')

In [ ]:
df_cal.head()

In [ ]:
# limit the data to 2016 only as there are few record in 2017
df_cal = df_cal[df_cal['date'] < '2017-01-01']

# convert the price to number
df_cal['price'] = df_cal[df_cal['price'].isnull() == False]['price'].apply(lambda x: float(x[1:].replace(',', '')))

In [ ]:
# check number of data in the year for a listing_id
df_cal.groupby(['listing_id']).count()['available'].unique()

# we have only 363 days of data, the first 3 days in Jan-2016 is missing

In [ ]:
# get the unavailabe count ('f') for each day
df_available_f = df_cal[df_cal['available'] == 'f'].groupby(['month']).count()['listing_id'].reset_index(name="count")
df_available_f.head(12)

In [ ]:
# get the availabe count ('t') for each day
# https://stackoverflow.com/questions/29836477/pandas-create-new-column-with-count-from-groupby
df_available_t = df_cal[df_cal['available'] == 't'].groupby(['month']).count()['listing_id'].reset_index(name="count")
df_available_t.head(12)

In [ ]:
df_cal['listing_id'].nunique()

### Number of Unavailable Listings in 2016

In [ ]:
# https://stackoverflow.com/questions/332289/how-do-you-change-the-size-of-figures-drawn-with-matplotlib
# https://stackoverflow.com/questions/44100429/plotting-period-series-in-matplotlib-pyplot
x = np.arange(0,len(df_available_f),1)
fig, ax = plt.subplots(1, 1, figsize=(12, 5))
ax.bar(x, df_available_f['count'])
ax.set_xticks(x)
ax.set_xticklabels(df_available_f['month'])
plt.xticks(rotation=45)
plt.title('Number of Unavailable Listings in 2016')
plt.show()

### Number of Available Listings in 2016

In [ ]:
x = np.arange(0,len(df_available_t),1)
fig, ax = plt.subplots(1, 1, figsize=(12, 5))
ax.bar(x, df_available_t['count'])
ax.set_xticks(x)
ax.set_xticklabels(df_available_t['month'])
plt.xticks(rotation=45)
plt.title('Number of Available Listings in 2016')
plt.show()

In [ ]:
df_available_t['count'] + df_available_f['count']

From the charts above, we can see the busiest period in **2016** is **Jan-2016** and **Jul-2016**

## Now the price. Plot some charts on the price changs during the year

In [ ]:
# average price for each day
df_day_price = df_cal[df_cal['available'] == 't'].groupby(['date']).mean()['price'].reset_index(name="avg_price")
df_day_price.head()

In [ ]:
x = np.arange(0,len(df_day_price),1)
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.plot(x, df_day_price['avg_price'])
ax.set_xticks(x)
ax.set_xticklabels(df_day_price['date'])
plt.title('Average price of listings in 2016')
plt.show()

In [ ]:
# average price by month
df_month_price = df_cal.groupby(['month']).mean()['price'].reset_index(name="avg_price")
df_month_price.head(12)

In [ ]:
x = np.arange(0,len(df_month_price),1)
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.bar(x, df_month_price['avg_price'])
ax.set_xticks(x)
plt.xticks(rotation=65)
ax.set_xticklabels(df_month_price['month'])
plt.title('Average Price of Listings in 2016')
plt.show()

# Let's study the price change in Jul-2016

In [ ]:
# average price for Jul-2016
#df_july = df_cal.query("available == 't' and date.month == 7")
df_july = df_cal[(df_cal['available'] == 't') & (df_cal['date'].dt.month == 7)]
df_july_price = df_july.groupby(['date']).mean()['price'].reset_index(name="avg_price")
df_july_price.head()

In [ ]:
x = np.arange(0,len(df_july_price),1)
fig, ax = plt.subplots(1, 1, figsize=(18, 8))
ax.bar(x, df_july_price['avg_price'])
ax.set_xticks(x)
ax.set_xticklabels(df_july_price['date'].dt.strftime('%Y-%m-%d'))
plt.xticks(rotation=70)
plt.title('Average price of listings in Jul-2016')
plt.show()

We can see the prices for 1-Jul, 2-Jul, 8-Jul, 9-Jul, 15-Jul, 16-Jul, 22-Jul, 23-Jul, 29-Jul and 30-Jul (Friday, Saturday) were higher than the prices on other days

In [ ]:
# average price by listings
df_month_price = df_cal.groupby(['listing_id']).mean()['price'].reset_index(name="avg_price").sort_values(by='avg_price', ascending=False)
df_month_price.head()

We can see the average price is the highest in Jul-2016

## Q2: Is there a general upward trend of both new Airbnb listings and total Airbnb visitors to Seattle?

In [ ]:
df_host_since = pd.to_datetime(df_listings['host_since']).dt.to_period('A').value_counts().rename_axis('host_since').reset_index(name='counts')
df_host_since = df_host_since[df_host_since['host_since'] < pd.to_datetime('2016-01-01').to_period('A')]
df_host_since = df_host_since.sort_values(by='host_since')
df_host_since.set_index("host_since", drop=True, inplace=True)
df_host_since.plot(kind='bar')
plt.title("No of Host Since")

In [ ]:
df_superhosts_count = df_listings['host_is_superhost'].value_counts()
df_superhosts_count.plot(kind ='bar')
plt.title("Superhosts in 2016")
plt.show()

In [ ]:
df_superhosts_count

In [ ]:
df_reviews_month = pd.to_datetime(df_reviews['date']).dt.to_period('A').value_counts().rename_axis('date').reset_index(name='counts')
df_reviews_month = df_reviews_month[df_reviews_month['date'] < pd.to_datetime('2016-01-01').to_period('A')]
df_reviews_month = df_reviews_month.sort_values(by='date')
df_reviews_month
df_reviews_month.set_index("date", drop=True, inplace=True)
df_reviews_month.plot(kind='bar')
plt.title("No of Reviews in Year")

##  Q3: How to be successful in Airbnb business?

In [ ]:
df_listings.sort_values(by=['review_scores_rating', 'number_of_reviews'], ascending=[True, False])
#df_listings.sort_values(by=['number_of_reviews'], ascending=[False])

In [ ]:
df_listings[df_listings['id'] == 8474294]

In [ ]:
df_listings.columns

In [ ]:
df_listings.groupby(['property_type', 'room_type','bed_type']).mean()[['review_scores_rating', 'review_scores_accuracy','review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value']]

In [ ]:
#df_listings.groupby(['room_type']).count()[['review_scores_rating']].plot(kind="bar", legend=False)
df_listings.groupby(['room_type']).count()[['id']].plot(kind="bar", legend=False)
plt.title("Listing Count of Different Room Types")
plt.xticks(rotation=70)

In [ ]:
df_listings.groupby(['room_type']).mean()[['review_scores_rating']].plot(kind="bar", legend=False)
plt.title("Average Ratings of Different Room Types")
plt.xticks(rotation=70)

In [ ]:
# neighbourhood_cleansed
df_listings.groupby(['neighbourhood_cleansed']).mean()[['review_scores_rating']].plot(kind="bar", legend=False, figsize=(20, 10))
plt.title("Average Ratings of Neighbourhood (Cleansed)")
plt.xticks(rotation=90)
print(df_listings.groupby(['neighbourhood_cleansed']).mean()[['review_scores_rating']])

In [ ]:
df_responsetime_rating = df_listings.groupby(['host_response_time']).mean()[['review_scores_rating']].sort_values('review_scores_rating')
df_responsetime_rating
df_responsetime_rating.plot(kind="bar", legend=False, figsize=(10, 6))
plt.title("Average Ratings of Different Response Time")
plt.xticks(rotation=70)

In [ ]:
df_listings.sum()[list(amenities)].sort_values(ascending=True).plot(kind ='barh', figsize=(15, 10))

In [ ]:
amenities_list = list(amenities)
with_amenity_avg_rating = []
without_amenity_avg_rating = []
for en in amenities_list:
    with_amenity_avg_rating.append(df_listings[df_listings[en] == 1].mean()['review_scores_rating'])
    without_amenity_avg_rating.append(df_listings[df_listings[en] == 0].mean()['review_scores_rating'])
        
df_amenities_rating = pd.DataFrame()
df_amenities_rating['amenity'] = amenities_list
df_amenities_rating['with_avg'] = with_amenity_avg_rating
df_amenities_rating['without_avg'] = without_amenity_avg_rating

df_amenities_rating.head()

In [ ]:
# reset index
# https://www.dataquest.io/blog/adding-axis-labels-to-plots-with-pandas-and-matplotlib/
df_amenities_rating.set_index("amenity", drop=True, inplace=True)
df_amenities_rating = df_amenities_rating.sort_values("with_avg", ascending=False)

In [ ]:
# find "major" differences
df_amenities_rating[abs(df_amenities_rating['with_avg'] - df_amenities_rating['without_avg']) > 2]

In [ ]:
df_amenities_rating.plot(kind="barh", figsize=(40, 40), use_index=True, fontsize=30)
plt.title("Average Ratings for Amenity in 2016", fontsize=30)
plt.show()